# Example usage from of the AUX API

### Requirements

- pycryptodome
- aiohttp

This notebook provides a high level overview of the AUX API and how to use it. The information presented in the markdown cells is a conteptual overview.

### Server address

Regions:
- Europe https://app-service-deu-f0e9ebbb.smarthomecs.de
- USA https://app-service-usa-fd7cc04c.smarthomecs.com

### API configuration

Extracted from APK
```python
LICENSE = 'PAFbJJ3WbvDxH5vvWezXN5BujETtH/iuTtIIW5CE/SeHN7oNKqnEajgljTcL0fBQQWM0XAAAAAAnBhJyhMi7zIQMsUcwR/PEwGA3uB5HLOnr+xRrci+FwHMkUtK7v4yo0ZHa+jPvb6djelPP893k7SagmffZmOkLSOsbNs8CAqsu8HuIDs2mDQAAAAA='
LICENSE_ID = '3c015b249dd66ef0f11f9bef59ecd737'
COMPANY_ID = '48eb1b36cf0202ab2ef07b880ecda60d'
```

In [1]:
import pprint
from aux_cloud import AuxCloudAPI

cloud = AuxCloudAPI(region='eu')

#### Required headers

All requests require the following headers:

```python
"Content-Type": "application/x-java-serialized-object",
"licenseId": LICENSE_ID,
"lid": LICENSE_ID,
"language": "en",
"appVersion": SPOOF_APP_VERSION,
"User-Agent": SPOOF_USER_AGENT,
"system": SPOOF_SYSTEM,
"appPlatform": SPOOF_APP_PLATFORM,
"loginsession": '<session>',
"userid": '<userid>',
```

Some requests require additional headers

### Login to the AUX API

POST `/account/login`

Additional headers:

```python
"timestamp": time.time(),
"token": md5(f'{request_body}{BODY_ENCRYPT_KEY}')
```

Body should be AES encrypted

```python
{
  "email": email,
  "password": sha1(f'{password}{PASSWORD_ENCRYPT_KEY}'),
  "companyid": COMPANY_ID,
  "lid": LICENSE_ID
}
```

In [ ]:
email = ''
password = ''
await cloud.login(email, password)

### Get list of your families/homes

#### POST `/appsync/group/member/getfamilylist`

```python
{
  'familyList': [
    {
      'cityCode': '',
      'companyId': '',
      'countryCode': '',
      'createTime': '2022-10-23 21:24:31',
      'createUser': '...',
      'description': '',
      'extend': '',
      'familyid': '...',
      'familylimit': 2,
      'grouptype': '',
      'icon': '',
      'master': '...',
      'name': '<base64 encoded name>_<epoch timestamp in milliseconds>',
      'orgname': '',
      'provinceCode': '',
      'spaceId': '',
      'userid': '',
      'version': '2022-12-06 01:48:29',
      'zoneInfo': ''
    }
  ]
}
```

In [ ]:
family_data = await cloud.list_families()
pprint.pprint(family_data)

### Get the list of available devices

Personal devices: POST `/appsync/group/dev/query?action=select`
Shared devices: POST `/appsync/group/sharedev/querylist?querytype=shared`

Additional headers:

```python
"familyid": '...'
```

Personal devices request Body:
```python
{"pids":[]}
```

Shared devices request Body:
```python
{"endpointId":""}
```

#### Response

Item data:
```python
{
  'attributes': None,
  'cookie': 'ey...',
  'createTime': '',
  'devSession': '...',
  'devicetypeFlag': 0,
  'discoverFamily': False,
  'endpointId': '...',
  'extend': '{"order":1}',
  'extern': '',
  'familyid': '...',
  'friendlyName': 'heat pump name',
  'gatewayId': '',
  'groupdevice': None,
  'icon': '...',
  'irData': '',
  'mac': 'ec:...',
  'moduleextend': '',
  'moduletype': 0,
  'order': 0,
  'plateformId': '',
  'productId': '000000000000000000000000c3aa0000',
  'resourcetag': '',
  'roomId': '...',
  'spaceOpenId': '',
  'state': 1,
  'subdevicenum': 0,
  'testAdd': False,
  'thirdEndpointId': '',
  'uniqueId': '',
  'userId': '',
  'v1moduleid': '...',
  'v1moduletype': 0,
  'vGroup': ''
}
```

Example structure for personal devices:
```python
{
  'endpoints': [
    { <Item data> }
  ]
}
```

Example structure for shared devices:
```python
{
  'shareFromOther': [{'devinfo': { <Item data> }}]
}
```

In [ ]:
all_devices = []
for family in family_data:
  your_devices_data = await cloud.list_devices(family['familyid'])
  shared_device_data = await cloud.list_devices(family['familyid'], shared=True)
  all_devices = all_devices + your_devices_data + shared_device_data

pprint.pprint(all_devices)

### Query device state (on/off)

POST `/device/control/v2/querystate`

Request body:
```python
queried_devices = [{
  "did": device_id,
  "devSession": dev_session
}]

body = {
  "directive": {
    "header": {
      "namespace": "DNA.QueryState",
      "name": "queryState",
      "interfaceVersion": "2",
      "senderId": "sdk",
      "messageId": f'{userid}-{timestamp}',
      "messageType": "controlgw.batch"
    },
    "payload": {
      "studata": queried_devices,
      "msgtype": "batch"
    }
  }
}
```

Response:
```python
{
  'event': {
    'payload': {
      'data': [{'did': '00000000000000000000xxxxxxxxx', 'state': 0}],
      'msg': 'ok',
      'status': 0
    }
  }
}
```

In [ ]:
pprint.pprint(await cloud.query_device_state(
    all_devices[0]['endpointId'],
    all_devices[0]['devSession']
))

### Fetching device parameters

#### Heat pump

```python
await cloud.get_device_params(
    all_devices[3],
    [
      # Empty list fetches all parameters
    ]
)
```

Special parameters not returned by the empty list parameters

```python
await cloud.get_device_params(
    all_devices[3],
    [
      'hp_water_tank_temp', # This needs to be queried separately
    ]
)
```

POST `/device/control/v2/sdkcontrol?license=<LICENSE>`

Body:

```python
cookie = json.loads(base64.b64decode(device['cookie'].encode()))

body = {
    "directive": {
        "header": {
          "namespace": "DNA.KeyValueControl",
          "name": "KeyValueControl",
          "interfaceVersion": "2",
          "senderId": "sdk",
          "messageId": f'{device['endpointId']}-{timestamp}',
        },
        "endpoint": {
            "devicePairedInfo": {
                "did": device['endpointId'],
                "pid": device['productId'],
                "mac": device['mac'],
                "devicetypeflag": device['devicetypeFlag'],
                "cookie": base64.b64encode(json.dumps({
                  "device": {
                      "id": cookie['terminalid'],
                      "key": cookie['aeskey'],
                      "devSession": device['devSession'],
                      "aeskey": cookie['aeskey'],
                      "did": device['endpointId'],
                      "pid": device['productId'],
                      "mac": device['mac'],
                  }
                }, separators=(',', ':')).encode()).decode()
            },
            "endpointId": device['endpointId'],
            "cookie": {},
            "devSession": device['devSession'],
        },
        "payload": {
            "act": "get",
            "params": [],
            "vals": []
        },
    }
}
```

#### Response

```python
{
  'ac_mode': 4,
  'ac_pwr': 1,
  'ac_temp': 330,
  'ecomode': 0,
  'hp_auto_wtemp': 9,
  'hp_fast_hotwater': 0,
  'hp_hotwater_temp': 360,
  'hp_pwr': 1,
  'qtmode': 1,
  'ver_old': 0
}

# Additional parameters body
{
  'ac_errcode1': 0,
  'err_flag': 0,
  'hp_water_tank_temp': 33
}
```


In [ ]:
# All parameters
pprint.pprint(await cloud.get_device_params(
    all_devices[0],
    []
))

# Ambient temperature
pprint.pprint(await cloud.get_device_params(
    all_devices[0],
    ['mode']
))

# Water tank temp
pprint.pprint(await cloud.get_device_params(
    all_devices[3],
    ['hp_water_tank_temp']
))

### Setting device parameters

This is the exact same as fetching device parameters, but with the `act` set to `set` and the `vals` set to the new values in order corresponding to the `params` property.

```python
pprint.pprint(await cloud.get_device_params(
    all_devices[3],
    ['hp_fast_hotwater'],
    [1]
))
```

In [ ]:
await cloud.get_device_params(
    all_devices[3],
    ['hp_fast_hotwater'],
    [1]
)